# NMR Inversion Recovery homework

<strong>Author(s):</strong> Seth D. Veenbaas, Jessica A. Nash, The Molecular Sciences Software Institute

<div class="alert alert-block alert-info"> 
<h2>Overview</h2>

<strong>Objective:</strong>

* Use Pandas and Scipy to proceed NMR data.

* Calculate the $T_1$ relaxation times.

* Determine if solvent conditions affect $T_1$ relaxation times.

</div>

## Inversion recovery experiment
The inversion-recovery experiment measures $T_1$ relaxation times of any nucleus. If the net magnetization is placed along the -z axis, it will gradually return to its equilibrium position along the +z axis at a rate governed by $T_1$. The equation governing this behavior as a function of the time t after its displacement is: 

$$ M_z(t) = M_{z,\text{eq}} \cdot \left(1 - 2e^{-t/T_1}\right) $$

The basic pulse sequence consists of an 180<sup>°</sup> pulse that inverts the magnetization to the -z axis. During the following delay, relaxation along the longitudinal plane takes place. Magnetization comes back to the original equilibrium z-magnetization. A 90<sup>°</sup> pulse creates transverse magnetization. The experiment is repeated for a series of delay values taken from a variable delay list. A 1D spectrum is obtained for each value of vd and stored in a pseudo 2D dataset. The longer the relaxation delay (d<sub>1</sub>) is, the more precise the T<sub>1</sub> measurement is. An ideal relaxation time (d<sub>1</sub>) can be calculated (aq = acquisition time):

 $$ d_1 + \text{aq} = 5 \cdot T_1 $$

 <br>

![image.png](../images/t1_relaxation_pulse_sequence.png)

More information:
https://imserc.northwestern.edu/downloads/nmr-t1.pdf



## Importing Required Libraries

First, let's import the python libraries/packages we need to work with the data.

In [57]:
import pandas as pd
import numpy as np
from scipy.optimize import curve_fit
import mnova

%config InteractiveShell.ast_node_interactivity = 'all'

Let's load, reformat, and view our data for an Ibuprofen inversion recovery experiment in chloroform (CDCl3).

In [ ]:
# Load the data from the CSV file
ibuprofen_cdcl3_inversion_data = pd.read_csv('data/Ibuprofen_CDCl3_1H_inversion_recovery.csv', header=1)

# Runs reformating function
ibuprofen_cdcl3_inversion_data = mnova.rename_columns(ibuprofen_cdcl3_inversion_data)

# Plot inversion recovery data
ibuprofen_cdcl3_inversion_data.plot(x='Time(s)')

# Display the inversion recovery dataframe
ibuprofen_cdcl3_inversion_data

## The Effect of Solvent on NMR $T_1$ Relaxation

NMR T₁ relaxation times are influenced by the choice of solvent due to differences in molecular tumbling, viscosity, and solvent-solute interactions.

Let's compare the differences in $T_1$ relaxation for Ibuprofen in deuterated chloroform (CDCl<sub>3</sub>) vs. deuterated DMSO (DMSO-d<sub>6</sub>).


<div class="alert alert-block alert-warning">
<h3>Step 1</h3>

Load, reformat, and view the inversion recovery data for ibuprofen in DMSO (DMSO-d<sub>6</sub>).

- Load data from `'data/Ibuprofen_DMSO_1H_inversion_recovery.csv'`
- Save the data to a variable named: `ibuprofen_dmso_inversion_data`

</div>

In [ ]:
# Load the data from the DMSO CSV file


# Runs reformating function


# Plot inversion recovery data


# Display the inversion recovery dataframe


Let's define the functions you'll need to calculate $T_1$!

In [60]:
# Define the inversion recovery model with the parameters (M, T1, and C)
def inversion_recovery_model(time, M, T1, C):
    return M * (1 - 2 * np.exp(-time / T1)) + C

# Define function to fit data to a inversion recovery model
def fit_relaxation_data(time_data, peak_data):
    # Initial guess for M, T1, and C
    initial_guess = [max(peak_data), 1.0, min(peak_data)]
    
    # Fit the curve
    param_optimal, _ = curve_fit(inversion_recovery_model, time_data, peak_data, p0=initial_guess)
    
    return param_optimal

<div class="alert alert-block alert-warning">
<h3>Step 2</h3>

We need to use following `for` loop for each of our data sets. Modify the code in the following cell to be a reusable function. Incorporate the follow:

#### 1. Function Name: `calc_t1`

#### 2. Parameters:

- `df` (DataFrame): Inversion recovery data.

#### 3. Function body (indented code):

```python
# Create an empty DataFrame for t1_data with columns 'Peak' and 'T1(s)'
t1_data = pd.DataFrame(columns=['Peak', 'T1(s)'])

# Calculate T1 for each peak and plot the fit
for column in df.columns[1:]:
    
    # Fit relaxation data to calculate T1
    time_data = df['Time(s)']
    peak_data = df[column]
    param_optimal = fit_relaxation_data(time_data, peak_data)
    T1 = param_optimal[1]
    
    # Add T1 time to the t1_data DataFrame
    new_row = pd.DataFrame({'Peak': [column], 'T1(s)': [T1]})
    t1_data = pd.concat([t1_data, new_row], ignore_index=True)
```

#### 4. Return:

- `t1_data` (DataFrame): $T_1$ relaxation times for every NMR peak.

</div>

### Here is what it looks like to define a function:
```python
def function_name(parameters):
    # Optional: explain what your function does in a Docstring
    """
    Docstring
    """
    # Function body (indented code)
    return output  # Optional: Return a result
```

In [61]:
# Calculate T1 times from data in a Dataframe


<div class="alert alert-block alert-warning">
<h3>Step 3</h3>

Use your `calc_t1` function to calculate $T_1$ times for ibuprofen in each of the solvent conditions and save the results to the variables:
* `ibuprofen_cdcl3_t1_data`
* `ibuprofen_dmso_t1_data`

Display the results.

</div>

In [ ]:
# Calculate T1 ibuprofen_cdcl3_inversion_data


# Calculate T1 ibuprofen_dmso_inversion_data


In [ ]:
# display ibuprofen_cdcl3_t1_data


<div class="alert alert-block alert-warning">
<h3>Step 4</h3>

Use the Pandas `.describe()` method to compare the $T_1$ times in CDCl<sub>3</sub> and DMSO-d<sub>6</sub>.

</div>

In [ ]:
# Describe the ibuprofen_cdcl3_t1_data Dataframe


In [ ]:
# Describe the ibuprofen_dmso_t1_data Dataframe


<div class="alert alert-block alert-warning">
<h3>Step 5</h3>

Which solvent condition increased $T_1$ relaxation times the most?

What was the lowest $T_1$ time for peak 7.1_ppm?

</div>

In [68]:
condition =
lowest_T1_time_7_1_ppm =